### Simple voltage divider

### Imports

In [1]:
from pathlib import Path
import py4spice as spi

### Set up paths dictionary

In [2]:
paths_dict: dict[str, Path] = {}           # define empty paths dictionary
paths_dict["project"] = Path("/workspaces/learn_ngspice/circuits/divider")
paths_dict["netlists"] = paths_dict["project"] / "netlists"
paths_dict["results"] = paths_dict["project"] / "results"
paths_dict["results"].mkdir(parents=True, exist_ok=True) # create results directory if it does not exist

paths_dict["ngspice"] = Path("/usr/bin/ngspice") # ngspice executable

# create simulation transcript file. If it exists, make sure it is empty
sim_tran_filename: Path = paths_dict["results"] / "sim_transcript.log"
if sim_tran_filename.exists():  # delete and recreate. this makes sure it's empty
    sim_tran_filename.unlink()
sim_tran_filename.touch()

### Set up netlists dictionary
The netlists objects are concatenated, along with the control instructions to make a single netlist to simulate.

In [3]:
netlists_dict: dict[str, spi.Netlist] = {}  # define empty netlists dictionary

# create these lines to use when concatenating to make full netlist
netlists_dict["blankline"] = spi.Netlist("")  # blank line for spacing
netlists_dict["title"] = spi.Netlist("* Simple Voltage Divider")
netlists_dict["end_line"] = spi.Netlist(".end")

# the main circuit ("device under test")
netlists_dict["dut"] = spi.Netlist(paths_dict["netlists"] / "dut.cir")

### Set up vectors dictionary
Vectors objects are used to define a set of circuit signals to create and display

In [4]:
vectors_dict: dict[str, spi.Vectors] = {}   # empty vectors dictionary

vectors_dict["vec_all"] = spi.Vectors("all")
vectors_dict["vec_in"] = spi.Vectors("in")
vectors_dict["vec_out"] = spi.Vectors("out")

### Analyses to run during the simulation process

In [5]:
list_of_analyses: list[spi.Analyses] = []  # start with an empty list

# 1st analysis: operating point
op1 = spi.Analyses(
    name="op1",
    cmd_type="op",
    cmd="op",
    vector=vectors_dict["vec_all"],
    results_loc=paths_dict["results"],
)
list_of_analyses.append(op1)

# For this example there is only one analysis. If there were more, we would define and append. 

### Create control section
This is the section that tells ngspice what operations to do

In [6]:
my_control = spi.Control()  # create 'my_control' object

# add all the analyses defined above into the control section
for analysis in list_of_analyses:
    my_control.insert_lines(analysis.lines_for_cntl())

# convert control section into a netlist object
netlists_dict["control1"] = spi.Netlist(str(my_control))

### Create a single netlist, ready for simulation

In [7]:
# concatenate all tne netlists to make top1 and add to netlist dict
netlists_dict["top"] = (
    netlists_dict["title"]
    + netlists_dict["blankline"]
    + netlists_dict["dut"]
    + netlists_dict["blankline"]
    + netlists_dict["control1"]
    + netlists_dict["blankline"]
    + netlists_dict["end_line"]
    + netlists_dict["blankline"]
    )

# write netlist to a file so ngspice can read it
top_filename: Path = paths_dict["netlists"] / "top.cir"
netlists_dict["top"].write_to_file(top_filename)


# here is the netlist in top.cir and will be used for simulation
print(netlists_dict["top"])

* simple voltage divider

v1 in 0 dc 10v
r1 in out 1k
r2 out 0 2k

.control
* timestamp: mon dec  1 01:03:05 2025
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
op
print line all > /workspaces/learn_ngspice/circuits/divider/results/op1.txt
quit
.endc

.end



### Simulate

In [8]:
# prepare simulate object and simulate
sim: spi.Simulate = spi.Simulate(
    ngspice_exe=paths_dict["ngspice"],
    netlist_filename=top_filename,
    transcript_filename=sim_tran_filename,
    name="sim1",
    timeout=5,
)
sim.run()  # run the Ngspice simulation

### Convert simulation results to usable format

In [9]:
# There's a list of sim_results in case there was more than one analysis for the simulation.
#   In this case there is only one analysis performed during simulation
sim_results: list[spi.SimResults] = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

### Print out op (operating point) results

In [10]:
# display results for operating point analysis
spi.print_section("Operating Point Results", sim_results[0].table_for_print())


--- Operating Point Results ---
in         10.000
out        6.667
v1#branch −3.333m

-------------------------------

